In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from linearmodels import PanelOLS
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import ResidualsPlot
from scipy import stats

In [2]:
joint = pd.read_excel('joint_snap.xlsx')

In [3]:
#convert the month column into correct form
joint['Month'] = pd.to_datetime(joint['Month'])
age = (joint['Month'] - joint['Inception Date'])/np.timedelta64(1,'M')
age = [int(i) for i in age]
joint['Age'] = age
#joint.dropna(subset=["esg_score"], inplace=True)

#convert the esb_l from string to float
esb_l = pd.DataFrame({'Equity Style Box (Long)':['Large Value', 'Large Growth', 'Large Blend', 'Mid Value', 'Mid Growth', 'Mid Blend','Small Value','Small Growth','Small Blend'], 'esb_l': [1, 2, 3, 4, 5, 6, 7, 8, 9]})
joint_new = joint.join(esb_l.set_index('Equity Style Box (Long)'), on='Equity Style Box (Long)')

#relative_flow is the size divided by flow value
flow = joint_new["Fund Flow"]
size = joint_new['Fund Size']
relative_flow = flow/size
joint_new['relative_flow'] = relative_flow

joint_new = joint_new.rename(columns={"Fund Sustainability": "esg_score"})

In [4]:
#build a new column, in which the relative_flow is the value of last month
flow_org = joint_new['relative_flow']
flow_nxt = [np.nan]*len(flow_org)

for i in range(3276):
    for j in range(1,92):
         flow_nxt[92*i+j-1] =flow_org[92*i+j]
            
joint_new['relative_flow_nxt'] = flow_nxt

In [ ]:
joint_new

In [5]:
#build a new column, performance * esg_score
#performance_score_1 = [joint_new['capm_Performance'][i] * joint_new['esg_score'] for i in range(len(joint_new))]
performance_score_1 = joint_new['capm_Performance']*joint_new['esg_score']

In [6]:
joint_new['esg_capm_performance'] = performance_score_1

In [13]:
joint_new[joint_new['relative_flow']<-5]

,FundId,Month,Fund Size,Fund Flow,esg_score,capm_Performance,performance_Three_Factor,performance_Four_Factor,Inception Date,Equity Style Box (Long),Age,size_log,esb_l,relative_flow,relative_flow_nxt,esg_capm_performance
3284,FS0000BXSI,2017-05-01,3.758907e+06,-3.662993e+07,54.78,-0.660566,-2.473056,-2.852894,2015-06-05,Large Blend,22,6.575062,3.0,-9.744835,-8.686338e-01,-36.185811
4107,FS00008KTX,2016-12-01,5.287703e+06,-2.835736e+07,44.69,-9.903688,-10.041754,-9.899113,2011-06-21,Large Value,65,6.723267,1.0,-5.362889,NaN,-442.595808
4463,FS0000B9CQ,2015-12-01,1.146167e+07,-7.958218e+07,46.17,-4.972695,-4.862340,-3.200667,2013-03-28,Mid Blend,32,7.059248,6.0,-6.943334,7.182545e-04,-229.589330
15700,FS00009O4V,2017-01-01,3.758907e+06,-6.605266e+07,NaN,1.780712,4.010159,4.174090,2012-05-15,NaN,55,6.575062,NaN,-17.572307,NaN,NaN
17684,FSUSA08QKY,2013-09-01,1.494594e+07,-1.034888e+08,47.64,-2.852903,-2.678175,-1.380724,2008-03-07,Mid Value,65,7.174523,4.0,-6.924207,2.193251e-02,-135.912283
18090,FS0000AOE7,2016-11-01,5.650778e+06,-8.347688e+07,46.78,-4.934681,-5.239671,-7.592631,2011-12-01,Large Value,59,6.752108,1.0,-14.772635,-2.289075e-02,-230.844376
22437,FSGBR0588G,2018-10-01,1.603597e+07,-1.034888e+08,60.42,NaN,NaN,NaN,1993-02-28,Large Blend,307,7.205095,3.0,-6.453542,-1.926476e-03,NaN
59601,FSGBR04U89,2018-06-01,1.225497e+07,-6.673953e+07,NaN,-1.031994,-1.538187,-1.769471,1993-10-15,Large Blend,295,7.088312,3.0,-5.445917,1.890340e-03,NaN
61938,FSUSA0B33J,2013-11-01,9.848815e+06,-6.720785e+07,44.88,-1.017888,-3.702678,-3.557188,2010-09-30,Large Blend,37,6.993384,3.0,-6.823953,8.638715e-01,-45.682816
65811,FS00009U1F,2014-08-01,3.758907e+06,-1.034888e+08,NaN,1.643861,1.051378,0.919980,2012-11-06,Mid Blend,20,6.575062,6.0,-27.531626,-2.232951e-02,NaN


In [7]:
#catagorize the month - 2012/01 to 2019/08, totoally 92 months
month = pd.Categorical(joint_new.Month)

#use the 'Month' to divide the dataset, the index is Fund ID
df = joint_new.set_index(['FundId', 'Month'])
df['Month'] = month

In [8]:
df

Fund Size     Fund Flow  esg_score  \
FundId     Month                                               
FS00008SZD 2012-01-01  2.609599e+08  4.379443e+07        NaN   
           2012-02-01  2.988403e+08  2.572761e+07        NaN   
           2012-03-01  3.071028e+08  1.558167e+06        NaN   
           2012-04-01  3.440745e+08  3.753147e+07        NaN   
           2012-05-01  3.771187e+08  4.877692e+07        NaN   
...                             ...           ...        ...   
FSUSA09WNF 2019-04-01  8.137770e+06 -1.487095e+03        NaN   
           2019-05-01  7.979041e+06  8.084123e+02        NaN   
           2019-06-01  8.651099e+06  2.060187e+03      40.91   
           2019-07-01  8.737163e+06 -2.267210e+03        NaN   
           2019-08-01  8.255823e+06  2.131805e+03        NaN   

                       capm_Performance  performance_Three_Factor  \
FundId     Month                                                    
FS00008SZD 2012-01-01          1.507391                  3.652442   
           2012-02-01          1.544497                  1.220171   
           2012-03-01          4.025839                  0.078026   
           2012-04-01          2.647697                  0.989935   
           2012-05-01         -0.066138                 -0.807304   
...                                 ...                       ...   
FSUSA09WNF 2019-04-01         -3.067829                 -3.546688   
           2019-05-01          3.666930                  4.017213   
           2019-06-01          4.207674                  3.772864   
           2019-07-01          1.134835                  4.536452   
           2019-08-01         -5.479327                 -3.326145   

                       performance_Four_Factor Inception Date  \
FundId     Month                                                
FS00008SZD 2012-01-01                 1.841484     2009-01-28   
           2012-02-01                 1.155288     2009-01-28   
           2012-03-01                 0.963080     2009-01-28   
           2012-04-01                 1.171659     2009-01-28   
           2012-05-01                -0.354775     2009-01-28   
...                                        ...            ...   
FSUSA09WNF 2019-04-01                -4.150616     2008-11-11   
           2019-05-01                 5.035920     2008-11-11   
           2019-06-01                 3.199166     2008-11-11   
           2019-07-01                 4.007588     2008-11-11   
           2019-08-01                -2.716709     2008-11-11   

                      Equity Style Box (Long)  Age  size_log  esb_l  \
FundId     Month                                                      
FS00008SZD 2012-01-01            Large Growth   35  8.416574    2.0   
           2012-02-01            Large Growth   36  8.475439    2.0   
           2012-03-01            Large Growth   37  8.487284    2.0   
           2012-04-01            Large Growth   38  8.536653    2.0   
           2012-05-01            Large Growth   39  8.576478    2.0   
...                                       ...  ...       ...    ...   
FSUSA09WNF 2019-04-01             Large Blend  124  6.910505    3.0   
           2019-05-01             Large Blend  125  6.901951    3.0   
           2019-06-01             Large Blend  126  6.937071    3.0   
           2019-07-01             Large Blend  127  6.941370    3.0   
           2019-08-01             Large Blend  128  6.916760    3.0   

                       relative_flow  relative_flow_nxt  esg_capm_performance  \
FundId     Month                                                                
FS00008SZD 2012-01-01       0.167821           0.086092                   NaN   
           2012-02-01       0.086092           0.005074                   NaN   
           2012-03-01       0.005074           0.109079                   NaN   
           2012-04-01       0.109079           0.129341                   NaN   
           2012-05-01       0.129341   

In [9]:
#capm Performance summary
exog_vars = ['esg_score','size_log', 'Age', 'capm_Performance', 'esg_capm_performance']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.relative_flow, exog, entity_effects=True, time_effects=True)
mod_nxt = PanelOLS(df.relative_flow_nxt, exog, entity_effects=True, time_effects=True)

res = mod.fit()
res_nxt = mod_nxt.fit()

/home/yuxuan/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/yuxuan/anaconda3/lib/python3.7/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [14]:
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:          relative_flow   R-squared:                        0.0042
Estimator:                   PanelOLS   R-squared (Between):             -1.3347
No. Observations:              155105   R-squared (Within):              -0.0018
Date:                Mon, Apr 06 2020   R-squared (Overall):             -0.2082
Time:                        20:54:11   Log-likelihood                 5.326e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      126.56
Entities:                        3261   P-value                           0.0000
Avg Obs:                       47.564   Distribution:                F(5,151749)
Min Obs:                       1.0000                                           
Max Obs:                       91.000   F-statistic (robust):             126.56
                            

In [15]:
print(res_nxt)

                          PanelOLS Estimation Summary                           
Dep. Variable:      relative_flow_nxt   R-squared:                        0.0039
Estimator:                   PanelOLS   R-squared (Between):             -7.3761
No. Observations:              152797   R-squared (Within):              -0.0426
Date:                Mon, Apr 06 2020   R-squared (Overall):             -1.0472
Time:                        20:54:12   Log-likelihood                 2.191e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      118.19
Entities:                        3251   P-value                           0.0000
Avg Obs:                       47.000   Distribution:                F(5,149452)
Min Obs:                       1.0000                                           
Max Obs:                       90.000   F-statistic (robust):             118.19
                            